In [ ]:
粘包：
1.只有TCP有粘包现象，UDP没有：因为TCP包不能判断包的边界
2.发送端优化传输，待缓冲区满再发送；接收端接收不及时，每次从缓冲区中取出旧包
3.recv(1024):每次从缓冲区取1024 bytes,
  send(1024):先放入缓冲区，再发送；如果1024》缓冲区剩余大小，就会造成数据丢失；
  sendall可以循环调用send发送数据，无数据丢失

In [ ]:
# 方案1：获取接收的字节流长度，循环接收全部数据流，即要获取包头信息
# 使用struct模块对打包的长度为固定4个字节或者八个字节，struct.pack.format参数是“i”时，只能打包长度为10的数字，那么还可以先将长度转化为json字符串，再打包。

# client
import socket
import struct

phone = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
phone.connect(('127.0.0.1',8880))

while True:
    cmd = input('Please input command>>:').strip()
    if not cmd : continue
    phone.send(cmd.encode('utf-8'))
    
    # 先接收包头，解包
    header_struct = phone.recv(4)
    unpack_res = struct.unpack('i', header_struct)
    total_size = unpack_res[0] # 包总长度
    
    # 接收数据流
    recv_size = 0
    total_data = b''
    while recv_size < total_size:
        recv_data = phone.recv(1024)
        recv_size += len(recv_data)
        total_data += recv_data
    print('response all :',total_data.decode('utf-8'))
phone.close()

# server
import socket
import subprocess
import struct

phone = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
phone.bind(('127.0.0.1',8880))
phone.listen(5) # 最大阻塞数

print('server is running')
while True:
    conn, addr = phone.accept()
    print(conn, addr)
    while True:
        res = subprocess.Popen(cmd.decode('utf-8'), shell=True, stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE)
        stdout = res.stdout.read()
        stderr = res.stderr.read()

        # 用 struct mudole 封装包头，转成固定长度的bytes data_len = len(stdout) + len(stderr)
        header = struct.pack('i', len(stdout) + len(stderr))
        conn.send(header)

        conn.send(stdout)
        conn.send(stderr)
    conn.close()
phone.close()

In [ ]:
# 方案2：优化包头信息
# 用字典描述发送的信息，然后json序列化为bytes格式，由于bytes格式的json字符串长度不固定，
# 所以要用struct模块将bytes格式的json字符串长度压缩成固定长度

# client
import socket
import struct
import json
phone = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
phone.connect(('127.0.0.1',8080)) #连接服务器
while True:
    # 发收消息
    cmd = input('请你输入命令>>：').strip()
    if not cmd:continue
    phone.send(cmd.encode('utf-8')) #发送
    #先收报头的长度
    header_len = struct.unpack('i',phone.recv(4))[0]  #吧bytes类型的反解
    #在收报头
    header_bytes = phone.recv(header_len) #收过来的也是bytes类型
    header_json = header_bytes.decode('utf-8')   #拿到json格式的字典
    header_dic = json.loads(header_json)  #反序列化拿到字典了
    total_size = header_dic['total_size']  #就拿到数据的总长度了
    #最后收数据
    recv_size = 0
    total_data=b''
    while recv_size<total_size: #循环的收
        recv_data = phone.recv(1024) #1024只是一个最大的限制
        recv_size+=len(recv_data) #有可能接收的不是1024个字节，或许比1024多呢，
        # 那么接收的时候就接收不全，所以还要加上接收的那个长度
        total_data+=recv_data #最终的结果
    print('返回的消息：%s'%total_data.decode('gbk'))
phone.close()

# server
import socket
import subprocess
import struct
import json
phone = socket.socket(socket.AF_INET,socket.SOCK_STREAM) #买手机
phone.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
phone.bind(('127.0.0.1',8080)) #绑定手机卡
phone.listen(5) #阻塞的最大数
print('start runing.....')
while True: #链接循环
    coon,addr = phone.accept()# 等待接电话
    print(coon,addr)
    while True: #通信循环
        # 收发消息
        cmd = coon.recv(1024) #接收的最大数
        print('接收的是：%s'%cmd.decode('utf-8'))
        #处理过程
        res = subprocess.Popen(cmd.decode('utf-8'),shell = True,
                                          stdout=subprocess.PIPE, #标准输出
                                          stderr=subprocess.PIPE #标准错误
                                )
        stdout = res.stdout.read()
        stderr = res.stderr.read()
        # 制作报头
        header_dic = {
            'total_size': len(stdout)+len(stderr),  # 总共的大小
            'filename': None,
            'md5': None
        }
        header_json = json.dumps(header_dic) #字符串类型
        header_bytes = header_json.encode('utf-8')  #转成bytes类型(但是长度是可变的)
        #先发报头的长度
        coon.send(struct.pack('i',len(header_bytes))) #发送固定长度的报头
        #再发报头
        coon.send(header_bytes)
        #最后发命令的结果
        coon.send(stdout)
        coon.send(stderr)
    coon.close()
phone.close()

In [5]:
# _*_ struct Module _*_ #
'''
      在网络通信当中，大多传递的数据是以二进制流（binary data）存在的。当传递字符串时，
  不必担心太多的问题，而当传递诸如int、char之类的基本数据的时候，就需要有一种机制将某些
  特定的结构体类型打包成二进制流的字符串然后再网络传输，而接收端也应该可以通过某种机制
  进行解包还原出原始的结构体数据。
  
      python中的struct模块就提供了这样的机制，该模块的主要作用就是对python基本类型值与
用python字符串格式表示的C struct类型间的转化
'''

# EG1.struct 用format specifier方式对数据进行打包和解包
# struct对象，format常用的有‘I3sf’，I 表示int，3s表示三个字符长度的字符串，f 表示 float
import struct
res = struct.pack('i',12345)
print(res, len(res), type(res)) # len=4

res2 = struct.pack('i',12345111)
print(res2,len(res2),type(res2)) # len=4

unpack_res = struct.unpack('i', res)
print(unpack_res, type(unpack_res))
print(unpack_res[0])

# EG2. struct 基本用法
import json, struct

header = {'file_size':1073741824000, 'file_name':'/path/a.txt', 
          'md5':'8f6fbf8347faa4924a76856701edb0f3'}

# 为了包头可传送，需要序列化并转为bytes
head_bytes = bytes(json.dumps(header),encoding='utf-8')

# struct 封装包的长度，用固定4个字节长度
head_len_bytes = struct.pack('i',len(head_bytes))

# client send
conn.send(head_len_bytes) # 先发包头的长度
conn.send(head_bytes)     # 包头 json/bytes
conn.sendall(data)        # file data

# server receive
head_len_bytes = s.recv(4) #先收包头长度字节
x = struct.unpack('i',head_len_bytes)[0] # 提取出长度值

head_bytes = s.recv(x)  # 接收包头信息
header = json.loads(json.dumps(header)) # 提取包头

# 接收数据
real_data_len = s.recv(header['file_size'])
s.recv(real_data_len)

b'90\x00\x00' 4 <class 'bytes'>
b'\x17_\xbc\x00' 4 <class 'bytes'>
(12345,) <class 'tuple'>
12345
